In [1]:
import os

from IPython.display import display, Markdown, Image

In [2]:
REPO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
TASK_PATH = os.path.join(REPO_PATH, "tasks", "02-structural-linguistics")
DATA_PATH = os.path.join(TASK_PATH, "data")

In [3]:
def show_markdown(path):
    with open(path, 'r') as fh:
        content = fh.read()
    display(Markdown(content))

In [4]:
show_markdown(os.path.join(TASK_PATH, "3-collocations.md"))

## Колокації

Одним із важливих аспектів будь-якої природної мови є сполучуваність слів. В англійській мові поняття сполучуваності дуже відчутне і присутнє у всіх мовних структурах: ми кажемо _"make a mistake",_ але _"do a favour";_ ми кажемо _"big surprise",_ але _"great anger";_ ми кажемо _"highly unlikely",_ але _"seriously wrong"._

### 1. Англійська мова

У цьому завданні вам потрібно дослідити сполучуваність дієслів одного синонімного ряду з прислівниками. Наприклад, ми частіше кажемо _"love somebody dearly", "honor somebody highly",_ але _"admire somebody greatly"._

**Завдання:**
1. продовжте синонімний ряд дієслів: _"say", "tell", "speak", "claim", "communicate"_
2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує
3. напишіть програму, яка знайде усі можливі прислівники для ваших дієслів у [корпусі блогів](data/blog2008.txt)
4. на виході програма повинна видати десять найчастотніших прислівників для кожного дієслова (разом із частотою). Приклад виводу:
	```
	say: (loudly, 51), (silently, 45), (quietly, 10)
	tell: (quietly, 100), (loudly, 61), (seriously, 5)
	```
5. збережіть програму та результати аналізу на корпусі (вивід) у директорії з вашим іменем

Синоніми можна знайти у тезаурусах (http://www.thesaurus.com/, https://www.merriam-webster.com/thesaurus) чи [WordNet](http://wordnetweb.princeton.edu/perl/webwn).

Підказка: зверніть увагу на те, що у природній мові дієслова можуть мати різні форми, а прислівників може бути по декілька.

### 2. Українська мова

У цьому завданні вам потрібно дослідити сполучуваність іменників-істот з прикметниками.

**Завдання:**
1. напишіть функцію, яка знаходить у реченні іменник, що позначає істоту, і витягає усі прикметники, якими цей іменник керує
2. напишіть програму, яка знайде усі можливі словосполучення прикметників та іменників-істот у [романі "Тигролови"](data/tyhrolovy.txt)
3. на виході програма повинна видати усі знайдені словосполучення та їх частоти. Приклад виводу:
	```
	84: старий сірко
	5: великий начальник
	```
4. збережіть програму та результати виводу в директорії з вашим іменем

Підказка: можна шукати послідовність прикметника та іменника і визначати словосполучення через pymorphy2. Також можна попрацювати з деревами залежностей через StanfordNLP, але можливо доведеться запускати код десь на Google Colab, бо там дуже великі моделі для української мови.

### Джерела

Ви можете використати будь-яку мову програмування та будь-яку NLP-бібліотеку.

Корпус блогів взятий з [Political Blog Corpora](http://www.cs.cmu.edu/~ark/blog-data/).


### Part 1

In [5]:
import re
import gc
import json
import spacy
import pandas as pd

from collections import Counter, OrderedDict
from functools import reduce
from multiprocessing import cpu_count
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [6]:
nlp_en = spacy.load('en_core_web_md')

In [7]:
synonyms = ["say", 
            "add", "announce", "answer", "assert", "claim", "convey", "declare", "deliver",
            "disclose", "do", "estimate", "express", "maintain", "mention", "read", "repeat", "reply",
            "report", "respond", "reveal", "speak", "state", "suggest", "tell", "voice", "affirm", 
            "allege", "communicate", "conjecture", "divulge", "flap", "gab", "guess", "imagine", 
            "imply", "jaw", "judge", "lip", "opine", "orate", "perform", "pronounce", "rap", "recite", 
            "rehearse", "relate", "remark", "render", "rumor", "spiel", "utter", "verbalize", "yak"]

In [8]:
def count_adv(sentence, verbs=synonyms):
    tokens = nlp_en(sentence)
    cnt = Counter()
    
    for token in tokens:
        if token.pos_ == 'ADV' and token.text[-2:] == 'ly' and token.dep_ == 'advmod':
            for ancestor in token.ancestors:
                if ancestor.lemma_ in verbs and ancestor.pos_ == 'VERB':
                    cnt.update({(ancestor.lemma_, token.lower_):1})
    return cnt

In [9]:
corpus_df = pd.read_table(os.path.join(DATA_PATH, 'blog2008.txt'), names=["text"])

In [10]:
%%time

try:
    with open('collocations_en.json', 'r') as file_:
        collocations = json.load(file_)
except FileNotFoundError:
    lst = process_map(count_adv, corpus_df.text.values, max_workers=cpu_count())
    total_cntr = reduce((lambda x, y: x + y), lst)

    collocations = {}
    for (verb, adv), count in total_cntr.items():
        collocations[verb] = collocations.get(verb, []) + [(adv, count)]
        
    collocations = {verb: sorted(cnt, key=lambda x: x[1], reverse=True)[:10] for verb, cnt in collocations.items()}
    
    with open('collocations_en.json', 'w') as file_:
        json.dump(collocations, file_, sort_keys=True, indent=4)
    
    del lst, total_cntr
    gc.collect()

CPU times: user 1.1 ms, sys: 85 µs, total: 1.19 ms
Wall time: 745 µs


In [11]:
OrderedDict(sorted(collocations.items()))

OrderedDict([('add',
              [['only', 38],
               ['really', 27],
               ['simply', 13],
               ['certainly', 11],
               ['actually', 10],
               ['probably', 9],
               ['quickly', 9],
               ['early', 8],
               ['currently', 7],
               ['fully', 5]]),
             ('affirm',
              [['simply', 3],
               ['repeatedly', 2],
               ['falsely', 2],
               ['constantly', 1],
               ['traditionally', 1],
               ['hopelessly', 1],
               ['clearly', 1]]),
             ('allege',
              [['only', 4],
               ['subsequently', 3],
               ['officially', 2],
               ['potentially', 2],
               ['personally', 2],
               ['flatly', 2],
               ['wrongly', 2],
               ['directly', 2],
               ['primarily', 1],
               ['intimately', 1]]),
             ('announce',
              [['recently', 1

### Part 2

In [12]:
import stanfordnlp
import warnings

warnings.filterwarnings("ignore")

In [13]:
nlp_uk = stanfordnlp.Pipeline(lang='uk')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': '

In [14]:
corpus_df = pd.read_table(os.path.join(DATA_PATH, 'tyhrolovy.txt'), names=["text"])

In [15]:
corpus_df

,text
0,Тигролови
1,Іван Багряний
2,ЧАСТИНА ПЕРША
3,Розділ перший
4,ДРАКОН
...,...
2653,"Гриць прип'яв Заливая, щоб не втік. Так звеліл..."
2654,Але Заливай скучав. Скучав смертельно. Вірний ...
2655,Подався доганяти без надії догнати.
2656,Але — сміливі завжди мають щастя.


In [16]:
def count_adj_uk(string):
    sentences = nlp_uk(string).sentences
    cnt = Counter()
    
    for sentence in sentences:
        for token in sentence.words:
            if token.upos == 'NOUN':
                res = [tkn.lemma.lower() for tkn in sentence.words 
                                 if tkn.upos == 'ADJ' and tkn.governor == int(token.index)]
                for _ in res:
                    cnt.update({(token.lemma.lower(), _):1})
    return cnt

In [17]:
# from torch.multiprocessing import Pool, Process, set_start_method
# try:
#      set_start_method('spawn', force=True)
# except RuntimeError:
#     pass

In [18]:
%%time

try:
    with open('collocations_uk.json', 'r') as file_:
        collocations = json.load(file_)
except FileNotFoundError:
#     lst = list(map(count_adj_uk, corpus_df.text.values))
    lst = []
    for item in tqdm(corpus_df.text.values):
        lst.append(count_adj_uk(item))
    total_cntr = reduce((lambda x, y: x + y), lst)

    collocations = {}
    for (verb, adv), count in total_cntr.items():
        collocations[verb] = collocations.get(verb, []) + [(adv, count)]
        
    collocations = {verb: sorted(cnt, key=lambda x: x[1], reverse=True)[:10] for verb, cnt in collocations.items()}
    
    with open('collocations_uk.json', 'w') as file_:
        json.dump(collocations, file_, sort_keys=True, indent=4, ensure_ascii=False)
    
    del lst, total_cntr
    gc.collect()

CPU times: user 10.7 ms, sys: 117 µs, total: 10.9 ms
Wall time: 9.98 ms


In [19]:
OrderedDict(sorted(collocations.items()))

OrderedDict([('абажур', [['різнокольоровий', 1], ['вибагливий', 1]]),
             ('абрикос', [['консервований', 2]]),
             ('авантюрник', [['індустріальний', 1]]),
             ('агонія', [['останній', 1], ['дитячий', 1]]),
             ('адреса', [['повний', 1]]),
             ('азіата', [['далекий', 1], ['дикий', 1]]),
             ('акомпанемент', [['монотонний', 1]]),
             ('аксесуар',
              [['дивогідний', 1],
               ['найдивоглядніший', 1],
               ['надзвичайний', 1]]),
             ('аксесуара', [['фантастичний', 1], ['екзотичний', 1]]),
             ('альфонс', [['кралі', 1]]),
             ('аматор', [['новітний', 1], ['пржевальський', 1]]),
             ('амбразура', [['маленький', 1]]),
             ('амур',
              [['широченний', 2],
               ['скований', 1],
               ['найвужчий', 1],
               ['стиснений', 1]]),
             ('амурець', [['другий', 1], ['здоровенний', 1]]),
             ('анісовка', [['вип